# Container

Cells must have a unique name.
If you change something in the cell, such as add a label, a grating coupler or just a pin and keep the same name it is very likely that you will have a name conflict when you combine two cells with the same name but different geometry.


Containers are an easy way to take an unchanged cell, then encapsulate it in a cell with a different name and add the old cell.
Most containers preserve the original cell name inside the cell.
For example, if you want to add padding to a Component called 'waveguide_L3', you can use the original name and just add a suffix.

`container_name = "f{component.name}_padding"`


if you don't define the port of the container, it will automatically take the ports from the component that it contains.
It will also encapsulate the component settings.

In [ ]:
from typing import Tuple, Optional,List
import pp


@pp.container
def add_padding(
    component: pp.Component,
    layers: List[Tuple[int, int]] = [pp.LAYER.DEVREC],
    suffix: str = "p",
    default: float = 5.0,
    top: Optional[float] = None,
    bottom: Optional[float] = None,
    right: Optional[float] = None,
    left: Optional[float] = None,

) -> pp.Component:
    """Adds padding layers to a container.

    Args:
        component
        layers: list of layers
        suffix for name
        default: default padding
        top: north padding
        bottom: south padding
        right: east padding
        left: west padding
    """

    container = pp.Component(name=f"{component.name}_{suffix}")
    container << component

    c = component
    top = top if top else default
    bottom = bottom if bottom else default
    right = right if right else default
    left = left if left else default

    points =   [[c.xmin - left, c.ymin - bottom],
        [c.xmax + right, c.ymin - bottom],
        [c.xmax + right, c.ymax + top],
        [c.xmin - left, c.ymax + top]]

    for layer in layers:
        container.add_polygon(points, layer=layer)
    return container

wg = pp.c.waveguide()
wg

In [ ]:
wg_padding = add_padding(wg)
wg_padding

In [ ]:
wg.settings

In [ ]:
wg_padding.settings

# useful containers

In [ ]:
import pp

c = pp.c.waveguide()
c

In [ ]:
pp.add_padding?

In [ ]:
c = pp.c.waveguide()
cc = pp.add_padding(c, default=5)
cc

In [ ]:
pp.add_pins?

In [ ]:
c = pp.c.waveguide()
cc = pp.add_pins(c)
cc

In [ ]:
cc = pp.extend_ports?

In [ ]:
cc = pp.extend_ports

In [ ]:
cc = pp.extend_ports

In [ ]:
c = pp.c.waveguide()
cc = pp.extend_ports(c)
cc

In [ ]:
c = pp.c.waveguide()
cc = pp.add_termination(c)
cc

In [ ]:
pp.routing.add_fiber_array?

In [ ]:
c = pp.c.waveguide()
cc = pp.routing.add_fiber_array(c)
cc

In [ ]:
pp.routing.add_fiber_single?

In [ ]:
c = pp.c.waveguide()
cc = pp.routing.add_fiber_single(c)
cc